<a href="https://colab.research.google.com/github/jakobengdahl/SCBDatahubDefinition/blob/main/DataEkosystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install networkx --quiet
!pip install yfiles_jupyter_graphs --quiet

In [20]:
# Import required libraries
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import HTML, VBox, HBox, Tab, Label, Button
import json
import pandas as pd
import numpy as np

In [21]:
from google.colab import output
#output.enable_custom_widget_manager()

In [22]:

def setup_ui(df):
    out = widgets.Output()
    with out:
        display(df)
    return out

def on_search_click(btn):
  url2 = 'https://raw.githubusercontent.com/jakobengdahl/SCBDatahubDefinition/main/ai_search.csv'
  df_ai = pd.read_csv(url2)
  df_ai = df_ai.replace(np.nan, '-')
  ai_tabell = setup_ui(df_ai)
  ai_tabell.layout.visibility = 'visible'
  ai.children=tuple(list(ai.children) + [ai_tabell])

url = 'https://raw.githubusercontent.com/jakobengdahl/SCBDatahubDefinition/main/artefakt.csv'
df = pd.read_csv(url)
df = df.replace(np.nan, '-')


In [31]:
# Create headline widgets
headline1 = HTML(value="<h2>Information om datahubben</h2>")
headline2 = HTML(value="<h2>Lägg till information om annan datahubb</h2>")
headline3 = HTML(value="<h2>Artefakter</h2>")

# Create widgets for text input fields and labels in the first form
benamning_label1 = Label("Benämning:")
benamning_input1 = widgets.Text()
organisatorisk_tillhorighet_label = Label("Organisatorisk tillhörighet:")
organisatorisk_tillhorighet_input = widgets.Text()

# Create widgets for text input fields and labels in the second form
benamning_label2 = Label("Benämning:")
benamning_input2 = widgets.Text()
sokvag_till_definition_label = Label("Sökväg till definition:")
sokvag_till_definition_input = widgets.Text()
sokvag_till_mer_info_label = Label("Sökväg till mer information om datahubben:")
sokvag_till_mer_info_input = widgets.Text()

# Create widgets for text input fields and labels in the third form
benamning_label3 = Label("Benämning:")
benamning_input3 = widgets.Text()
beskrivning_label = Label("Beskrivning:")
beskrivning_input = widgets.Text()
licens_label = Label("Licens:")
licens_input = widgets.Text()
tillganglighetsgrad_label = Label("Tillgänglighetsgrad:")
tillganglighetsgrad_input = widgets.Text()
kontaktvag_label = Label("Kontaktväg:")
kontaktvag_input = widgets.Text()
nyckelord_label = Label("Nyckelord:")
nyckelord_input = widgets.Text()
skapat_datum_label = Label("Skapat datum:")
skapat_datum_input = widgets.Text()
beroenden_label = Label("Beroenden:")
beroenden_input = widgets.Text()
sokvag_label = Label("Sökväg:")
sokvag_input = widgets.Text()

# Create "Lägg till" buttons for the second and third forms
lagg_till_button2 = Button(description="Lägg till")
lagg_till_button3 = Button(description="Lägg till")

# Create a VBox to hold the connected datahub labels
connected_datahub_labels = VBox([])

# Create a VBox to hold the artifact labels
artifact_labels = VBox([])

# Create VBox layouts for each form
form1 = VBox([headline1, benamning_label1, benamning_input1, organisatorisk_tillhorighet_label, organisatorisk_tillhorighet_input])
form2 = VBox([headline2, benamning_label2, benamning_input2, sokvag_till_definition_label, sokvag_till_definition_input, sokvag_till_mer_info_label, sokvag_till_mer_info_input, lagg_till_button2, connected_datahub_labels])
form3 = VBox([headline3, benamning_label3, benamning_input3, beskrivning_label, beskrivning_input, licens_label, licens_input, tillganglighetsgrad_label, tillganglighetsgrad_input, kontaktvag_label, kontaktvag_input, nyckelord_label, nyckelord_input, skapat_datum_label, skapat_datum_input, beroenden_label, beroenden_input, sokvag_label, sokvag_input, lagg_till_button3, artifact_labels ])

# Create Tab widget and set tab titles for the inner tab
inner_tab = Tab(children=[form1, form2, form3])
inner_tab.set_title(0, "Information om datahubben")
inner_tab.set_title(1, "Lägg till information om annan datahubb")
inner_tab.set_title(2, "Artefakter")


# tab 3: hitta resurs
hitta_headline = HTML(value="<h2>Hitta artefakter</h2>")
hitta_tabell = setup_ui(df)
hitta = VBox([hitta_headline, hitta_tabell])

ai_headline = HTML(value="<h2>AI matching</h2>")
ai_beskrivning_label = Label("Ange en beskrivning:")
ai_beskrivning_input = widgets.Textarea(layout=widgets.Layout(height="100px", width="50%"))
ai_sok = Button(description="Sök")
ai_sok.on_click(on_search_click)
ai = VBox([ai_headline, ai_beskrivning_label, ai_beskrivning_input, ai_sok])#, ai_tabell])

tab_content_3 = Tab(children=[hitta, ai])
tab_content_3.set_title(0, "Hitta artefakter")
tab_content_3.set_title(1, "AI matchning")


# Create placeholders for the additional tabs
#tab_content_3 = widgets.HTML("Tab content 3")
tab_content_4 = widgets.HTML("Tab content 4")




In [32]:
# Initialize the datahubdefinition_data dictionary
datahubdefinition_data = {
    "datahubdefinition": {
        "name": "",
        "organizational_affiliation": "",
        "connected_datahubs": [],
        "artifacts": []
    }
}

# Function to add a new connected datahub
def add_connected_datahub(button):
    connected_datahub = {
        "name": benamning_input2.value,
        "definition_path": sokvag_till_definition_input.value,
        "more_info_path": sokvag_till_mer_info_input.value
    }
    
    datahubdefinition_data["datahubdefinition"]["connected_datahubs"].append(connected_datahub)
    
    # Create a label for the new connected datahub and add it to the connected_datahub_labels VBox
    new_label = Label(connected_datahub["name"])
    connected_datahub_labels.children += (new_label,)

In [33]:
# Function to add a new artifact
def add_artifact(button):
    artifact = {
        "name": benamning_input3.value,
        "description": beskrivning_input.value,
        "license": licens_input.value,
        "accessibility_level": tillganglighetsgrad_input.value,
        "contact_path": kontaktvag_input.value,
        "keywords": nyckelord_input.value,
        "created_date": skapat_datum_input.value,
        "dependencies": beroenden_input.value,
        "path": sokvag_input.value
    }
    
    datahubdefinition_data["datahubdefinition"]["artifacts"].append(artifact)
    
    # Create a label for the new artifact and add it to the artifact_labels VBox
    new_label = Label(artifact["name"])
    artifact_labels.children += (new_label,)

# Attach the add_artifact function to the "lagg_till_button3" button click event
lagg_till_button3.on_click(add_artifact)

In [34]:
w = GraphWidget()
w.nodes = [
    {"id": 0, "properties": {"label": "Skatteverket", "url": "https://www.skatteverket.se"}},
    {"id": 1, "properties": {"label": "Bolagsverket", "url": "https://www.bolagsverket.se"}},
    {"id": 2, "properties": {"label": "Trafikverket", "url": "https://www.trafikverket.se"}},
    {"id": 3, "properties": {"label": "Försäkringskassan", "url": "https://www.forsakringskassan.se"}},
    {"id": 4, "properties": {"label": "Arbetsförmedlingen", "url": "https://www.arbetsformedlingen.se"}},
    {"id": 5, "properties": {"label": "Statistiska Centralbyrån", "url": "https://www.scb.se"}},
    {"id": 6, "properties": {"label": "Sveriges Riksbank", "url": "https://www.riksbank.se"}},
    {"id": 7, "properties": {"label": "Socialstyrelsen", "url": "https://www.socialstyrelsen.se"}}
]

w.edges = [
    {"id": 0, "start": 0, "end": 1, "properties": {"label": "data"}},
    {"id": 1, "start": 1, "end": 2, "properties": {"label": "technical artefacts"}},
    {"id": 2, "start": 2, "end": 3, "properties": {"label": "experts exchange"}},
    {"id": 3, "start": 3, "end": 4, "properties": {"label": "data"}},
    {"id": 4, "start": 4, "end": 5, "properties": {"label": "technical artefacts"}},
    {"id": 5, "start": 5, "end": 6, "properties": {"label": "experts exchange"}},
    {"id": 6, "start": 6, "end": 7, "properties": {"label": "data"}},
    {"id": 7, "start": 0, "end": 7, "properties": {"label": "experts exchange"}},
    {"id": 8, "start": 0, "end": 3, "properties": {"label": "technical artefacts"}},
    {"id": 9, "start": 1, "end": 3, "properties": {"label": "experts exchange"}},
    {"id": 10, "start": 2, "end": 4, "properties": {"label": "data"}},
    {"id": 11, "start": 3, "end": 6, "properties": {"label": "technical artefacts"}},
    {"id": 12, "start": 4, "end": 7, "properties": {"label": "experts exchange"}},
    {"id": 13, "start": 0, "end": 6, "properties": {"label": "data"}},
    {"id": 14, "start": 1, "end": 4, "properties": {"label": "technical artefacts"}},
    {"id": 15, "start": 2, "end": 5, "properties": {"label": "experts exchange"}},
    {"id": 16, "start": 3, "end": 7, "properties": {"label": "data"}}
]

In [35]:
# Assuming you have the GraphWidget class imported and created an instance named 'w'
# Create the content for the "Hitta artefakter" tab
hitta_artefakter_content = HTML("Tab content 3") #VBox([w])

In [36]:
# Attach the add_connected_datahub function to the "lagg_till_button2" button click event
lagg_till_button2.on_click(add_connected_datahub)


# Create the "Spara" button
spara_button = Button(description="Spara")


# Define the save_datadefinition function
def save_datadefinition(button):
    # Read data from form1 and update the JSON structure
    datahubdefinition_data["datahubdefinition"]["name"] = benamning_input1.value
    datahubdefinition_data["datahubdefinition"]["organizational_affiliation"] = organisatorisk_tillhorighet_input.value

    # Save the JSON structure to a file called "DatahubDefinition.json"
    with open("DatahubDefinition.json", "w") as outfile:
        json.dump(datahubdefinition_data, outfile, indent=2)

    # Create a download link for the JSON file using Google Colab
    files.download('DatahubDefinition.json')

    # Update the buttons HBox to include the download link
    buttons.children = (spara_button)

    print("Data definition saved.")

# Attach the save_datadefinition function to the button's click event
spara_button.on_click(save_datadefinition)



# Place the "Spara" and "Exportera" buttons in an HBox
buttons = spara_button

# Create a new VBox that contains the inner_tab and the buttons HBox
inner_tab_with_buttons = VBox([inner_tab, buttons])

# Create the outer Tab widget and set tab titles
outer_tab = Tab(children=[inner_tab_with_buttons, hitta_artefakter_content, tab_content_3, tab_content_4])
outer_tab.set_title(0, "Skapa datahubbdefinition")
outer_tab.set_title(1, "Dataekosystem")
outer_tab.set_title(2, "Hitta artefakter")
outer_tab.set_title(3, "Använd artefakter")



In [37]:
# Display the outer tab layout
display(outer_tab)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Data definition saved.
